In [1]:
from itertools import islice
import openpyxl
from pandas import DataFrame
import os
import traceback
import datetime
import matplotlib.pyplot as plt

%matplotlib inline

In [37]:
# Convert a worksheet with headers to a Pandas dataframe
def ws2df(ws: openpyxl.worksheet) -> DataFrame:
    data = ws.values
    cols = next(data)[1:]
    data = list(data)
    idx = [r[0] for r in data]
    data = (islice(r, 1, None) for r in data)
    df = DataFrame(data, index=idx, columns=cols)
    return df

In [3]:
# Converts a 1-based column number to its letter
# Modified from https://stackoverflow.com/a/23862195/5139284
def colnum2str(n: int) -> str:
    string = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string

In [26]:
def histogram(filename: str, date_col_title: str = 'Exam date', by_year_not_month=False):
    print('Creating a histogram for ' + filename + '...')
    wb = openpyxl.load_workbook(filename)
    sheet = wb.worksheets[0]
    col_indices = {cell.value: n + 1 for n, cell in enumerate(list(sheet.rows)[0])}
    date_col = colnum2str(col_indices[date_col_title]) # e.g., 'B', 'D'
    
    dates_replaced: int = 0
    
    for row in range(2, sheet.max_row + 1):
        row = str(row)
        try:
            if sheet[date_col + row].value >= datetime.datetime(2100, 12, 31):
                sheet[date_col + row] = datetime.datetime(2100, 12, 31)
                dates_replaced += 1
        except AttributeError:
            print(f'Row {row} skipped due to an exception:')
            traceback.print_exc()
            continue
    print(dates_replaced, 'dates replaced')
    
    """
    i = 1
    iterations = 0
    while i <= sheet.max_column and iterations < 20:
        if sheet[colnum2str(i) + '1'].value == '':
            break
        elif sheet[colnum2str(i) + '1'].value != date_col_title:
            print('deleted ' + sheet[colnum2str(i) + '1'].value)
            sheet.delete_cols(i)
        else:
            i += 1
        iterations += 1
        print([col_title.value for col_title in list(sheet.rows)[0]])
    """
            
    
    df = ws2df(sheet)
    try:
        df[date_col_title] = df[date_col_title].astype('datetime64')
    except KeyError:
        date_col_title = df.columns[0]
        df[date_col_title] = df[date_col_title].astype('datetime64')
    # https://stackoverflow.com/a/29036738/5139284
    df.groupby([df[date_col_title].dt.year, df[date_col_title].dt.month]).count().plot(kind="bar")
    plt.savefig(f'{filename}_histogram.png')

In [28]:
histogram(os.path.join(r'C:\Users\micha\Google Drive\Patients 8-16-19', 'Insurance.xlsx'), date_col_title='DOB')

Creating a histogram for C:\Users\micha\Google Drive\Patients 8-16-19\Insurance.xlsx...


KeyboardInterrupt: 

In [42]:
filename = os.path.join(r'C:\Users\micha\Google Drive\Patients 8-16-19', 'Insurance.xlsx')
date_col_title = 'DOB'
print('Creating a histogram for ' + filename + '...')
wb = openpyxl.load_workbook(filename)
sheet = wb.worksheets[0]
col_indices = {cell.value: n + 1 for n, cell in enumerate(list(sheet.rows)[0])}
date_col = colnum2str(col_indices[date_col_title]) # e.g., 'B', 'D'

Creating a histogram for C:\Users\micha\Google Drive\Patients 8-16-19\Insurance.xlsx...


In [30]:
dates_replaced: int = 0

for row in range(2, sheet.max_row + 1):
    row = str(row)
    try:
        if sheet[date_col + row].value >= datetime.datetime(2100, 12, 31):
            sheet[date_col + row] = datetime.datetime(2100, 12, 31)
            dates_replaced += 1
    except AttributeError:
        print(f'Row {row} skipped due to an exception:')
        traceback.print_exc()
        continue
print(dates_replaced, 'dates replaced')


59 dates replaced


In [31]:
i = 1
iterations = 0
while i <= sheet.max_column and iterations < 20:
    if sheet[colnum2str(i) + '1'].value == '':
        break
    elif sheet[colnum2str(i) + '1'].value != date_col_title:
        print('deleted ' + sheet[colnum2str(i) + '1'].value)
        sheet.delete_cols(i)
    else:
        i += 1
    iterations += 1
    print([col_title.value for col_title in list(sheet.rows)[0]])



deleted ID
['Last Name', 'First Name', 'DOB', 'Insurance ID', 'Insurance ID 2', 'Can Call', 'Called', 'concat', 'concat', 's', 's']
deleted Last Name
['First Name', 'DOB', 'Insurance ID', 'Insurance ID 2', 'Can Call', 'Called', 'concat', 'concat', 's', 's']
deleted First Name
['DOB', 'Insurance ID', 'Insurance ID 2', 'Can Call', 'Called', 'concat', 'concat', 's', 's']
['DOB', 'Insurance ID', 'Insurance ID 2', 'Can Call', 'Called', 'concat', 'concat', 's', 's']
deleted Insurance ID
['DOB', 'Insurance ID 2', 'Can Call', 'Called', 'concat', 'concat', 's', 's']
deleted Insurance ID 2
['DOB', 'Can Call', 'Called', 'concat', 'concat', 's', 's']
deleted Can Call
['DOB', 'Called', 'concat', 'concat', 's', 's']
deleted Called
['DOB', 'concat', 'concat', 's', 's']
deleted concat
['DOB', 'concat', 's', 's']
deleted concat
['DOB', 's', 's']
deleted s
['DOB', 's']
deleted s
['DOB']


In [41]:
df = ws2df(sheet)
print(df)
print(df.head())
try:
    df[date_col_title] = df[date_col_title].astype('datetime64')
    # https://stackoverflow.com/a/29036738/5139284
    df.groupby([df[date_col_title].dt.year, df[date_col_title].dt.month]).count().plot(kind="bar")
except KeyError:
    df = df.astype('datetime64')
    df.groupby([df.dt.year, df.dt.month]).count().plot(kind="bar")

plt.savefig(f'{filename}_histogram.png')

fig, ax = plt.subplots()

Empty DataFrame
Columns: []
Index: [1952-04-15 00:00:00, 1985-05-19 00:00:00, 1977-02-20 00:00:00, 2011-05-21 00:00:00, 1960-03-10 00:00:00, 1956-06-05 00:00:00, 1965-07-11 00:00:00, 1963-01-10 00:00:00, 1934-05-22 00:00:00, 1993-02-02 00:00:00, 1964-08-04 00:00:00, 1990-09-25 00:00:00, 2011-03-09 00:00:00, 1931-02-20 00:00:00, 1981-09-25 00:00:00, 1989-01-06 00:00:00, 1984-03-03 00:00:00, 1980-01-31 00:00:00, 2028-10-05 00:00:00, 1961-09-07 00:00:00, 2007-07-05 00:00:00, 2003-06-24 00:00:00, 1964-12-30 00:00:00, 1945-04-20 00:00:00, 2007-03-28 00:00:00, 1985-12-31 00:00:00, 2005-09-14 00:00:00, 1975-04-06 00:00:00, 1974-03-29 00:00:00, 1949-08-25 00:00:00, 1992-01-31 00:00:00, 2004-07-10 00:00:00, 1936-06-29 00:00:00, 1939-06-28 00:00:00, 2010-10-06 00:00:00, 2004-03-08 00:00:00, 1960-11-22 00:00:00, 1949-11-01 00:00:00, 1977-01-31 00:00:00, 2009-08-04 00:00:00, 1982-05-28 00:00:00, 2006-08-29 00:00:00, 1973-01-20 00:00:00, 1967-03-07 00:00:00, 2003-11-28 00:00:00, 2028-06-03 00:00:00

AttributeError: 'DataFrame' object has no attribute 'dt'